In [1]:
# 為了美觀 我把Future Warning關閉
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import gdown
output = "pttsmalldata.zip"
id = "1BT4h4-kzrtCS_52P2i7C1rlj1GZgEbs6"
gdown.download(id=id, output=output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1BT4h4-kzrtCS_52P2i7C1rlj1GZgEbs6
To: /content/pttsmalldata.zip
100%|██████████| 49.2M/49.2M [00:00<00:00, 58.8MB/s]


'pttsmalldata.zip'

In [5]:
# 把 zip 解壓縮到 data 資料夾
import zipfile
import os
if not os.path.exists("data"):
    f = zipfile.ZipFile("./pttsmalldata.zip")
    f.extractall("./data")
    f.close()

In [6]:
import glob
import json
import pandas as pd
# 拿取所有 *.json 檔案
fs = glob.glob("./data/pttsmalldata/*.json")
# 我們只取用 內文 標題 和 型態(問卦...等等)
# 但實際上我們只使用 內文 做訓練而已
contents = []
titles = []
types = []
for fn in fs:
    with open(fn, "r", encoding="utf-8") as f:
        data = json.load(f)
        contents.append(data["post_content"])
        titles.append(data["post_title"])
        types.append(data["post_type"])
# 準備成表格
df = pd.DataFrame({
    "type":types,
    "title":titles,
    "content":contents
}, columns=["type", "title", "content"])
df

,type,title,content
0,問卦,萬一被女同事看到姆咪 該怎麼解釋？,\n\n如題\n\n只是萬一啦\n\n真的有那麼一個萬一\n\n被女同事看到我手機裡的姆咪圖...
1,新聞,比基尼女郎蘭嶼海邊戲水 被po網指褻瀆,\n自由時報\n\n比基尼女郎蘭嶼海邊戲水 被po網指褻瀆文化 引發熱議\n\n2019-0...
2,新聞,BBC介紹布袋戲大師陳錫煌 致力傳統藝術年,\n中央社\n\nBBC介紹布袋戲大師陳錫煌 致力傳統藝術年輕化\n\nhttps://ww...
3,問卦,報案怎麼那麼麻煩？,\n如標題\n\n陪朋友去報案\n\n順便自己報案\n\n發現啊\n\n去警察局報案時\n\...
4,問卦,受黨國教育的老人喜歡罵大學生素質低？,\n\n就是啊\n\n受黨國教育的9.2韓粉老人，超喜歡罵現在年輕人素質低落的耶！\n\n在...
...,...,...,...
19995,問卦,禿子為什麼都會禿到後腦勺？,\n本木村從高中開始\n就留著一頭長捲髮\n這輩子沒有禿頭的煩惱\n家族基因都沒人禿頭\n\...
19996,新聞,選舉激情過後 明年電價漲不漲？,\n2019-09-18 20:02經濟日報 記者江睿智╱即時報導\n\n如外界預期，經濟部...
19997,問卦,日本有人提議老人應放棄選舉權,\n\n莫邦富的日本管窺（258)日本有人提議老人應放棄選舉權\n\n2019/07/26\...
19998,問卦,頂呱呱哪一樣最好吃?,\n明天頂瓜瓜單品8折\n\n小魯很少吃，一年就吃這一次\n\n已經煩惱到晚餐吃不下東西了\...


In [7]:
# 使用 jieba 進行分詞
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import jieba
import os
from urllib.request import urlretrieve

# 使用大型字典
big_dict_path = "dict.txt.big"
if not os.path.exists(big_dict_path):
    print("下載大型字典")
    url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
    urlretrieve(url, big_dict_path)
jieba.set_dictionary(big_dict_path)

# 需加入一些鄉民常用字彙
ptt_dict_path = "ptt_dic.txt"
if os.path.exists(ptt_dict_path):
    print("載入ptt專用詞典")
    jieba.load_userdict(ptt_dict_path)

下載大型字典


In [8]:
punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…~/ －＊➜■─★☆=@<>◉é''')

In [9]:
import re
# 對表格的每一筆都做出轉換
def process(content):
    content = re.sub(r'https?:\/\/.*[\r\n]*', '', content)
    fil = filter(lambda x: x not in punct, jieba.cut(content))
    content = " ".join(fil)
    return content

process(df["content"][0])

Building prefix dict from /content/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.573 seconds.
DEBUG:jieba:Loading model cost 1.573 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


'\n \n 如 題 \n \n 只是 萬一 啦 \n \n 真的 有 那麼 一個 萬一 \n \n 被 女同事 看到 我 手機 裡的姆咪圖 的話 \n \n 該 怎麼 解釋 比較 內行 啊 \n \n \n 總 不能 尷尬 的 馬上 收起 來說 沒事 吧 \n \n 一聽 就 覺得 很 可疑 啊 \n \n 根本 是 下下策 \n \n \n 有沒有 類似 八卦 啊 \n \n 幹姆咪 \n \n -- \n 因為 太 可愛 了 嗎 \n \n 不好 吧 大概 講了 以後 就 不會 對話 了 \n ※ 編輯 david3809 1.75 214.180 日本 09 22 2019 20 11 34 \n 刷 一排 兔 外套 \n ※ 編輯 david3809 1.75 214.180 日本 09 22 2019 20 12 36 \n'

In [10]:
content_cut = df["content"].apply(process)
content_cut

0        \n \n 如 題 \n \n 只是 萬一 啦 \n \n 真的 有 那麼 一個 萬一 \n...
1        \n 自由 時報 \n \n 比基尼 女郎 蘭嶼 海邊 戲水 被 po 網指 褻瀆 文化 引...
2        \n 中央社 \n \n BBC 介紹 布袋戲 大師 陳 錫煌 致力 傳統 藝術 年輕化 \...
3        \n 如 標題 \n \n 陪 朋友 去 報案 \n \n 順便 自己 報案 \n \n 發...
4        \n \n 就是 啊 \n \n 受 黨國 教育 的 9.2 韓粉 老人 超 喜歡 罵 現在...
                               ...                        
19995    \n 本 木村 從 高中 開始 \n 就 留著 一頭 長捲髮 \n 這輩子 沒有 禿頭 的 ...
19996    \n 2019 09 18 20 02 經濟日報 記者 江 睿智 ╱ 即時 報導 \n \n...
19997    \n \n 莫邦富 的 日本 管窺 258 日本 有人 提議 老人 應 放棄 選舉權 \n ...
19998    \n 明天 頂瓜瓜 單品 8 折 \n \n 小魯 很少 吃 一年 就 吃 這 一次 \n ...
19999    \n 以下 從 維基百科 搬來 der \n \n \n \n 莎朗蒂 Sharon Tat...
Name: content, Length: 20000, dtype: object

In [11]:
df["cut"] = content_cut
df.to_csv("pttsmall.csv", encoding="utf-8", index=False)

In [12]:
f = open("pttsmalljoin.txt", "w", encoding="utf-8")
for content in content_cut:
    f.write("{}\n".format(content))
f.close()

In [13]:
from gensim.models import word2vec
sentences = word2vec.LineSentence('pttsmalljoin.txt')
model = word2vec.Word2Vec(sentences)

CPU times: user 1min 8s, sys: 776 ms, total: 1min 9s
Wall time: 46.2 s


In [14]:
model.wv["版主"]

array([ 0.3096566 , -0.7573841 ,  1.3167441 ,  0.42162317,  0.07933115,
       -0.31963336,  0.8668903 , -0.28346744, -0.34594658, -0.94586945,
        0.23109365, -0.08907744, -1.2102085 ,  1.4222153 ,  0.40473098,
        0.25679567,  0.64195687, -0.26492053,  0.38326427,  0.01758704,
        0.5619465 ,  0.70135725,  0.5169445 , -0.46652156,  0.5019666 ,
       -0.03671306, -0.4837524 ,  0.50529975,  0.049697  ,  0.20701374,
        1.4746095 ,  0.8509607 , -1.0045843 , -0.1741743 , -0.12620515,
       -0.86265135,  0.77709025,  1.4628671 , -0.24317203, -0.47014385,
       -0.2639367 ,  0.30751365,  0.50015485, -1.0780277 ,  1.6302983 ,
       -1.5249598 , -0.5266975 ,  0.7534834 ,  0.41034675,  0.5826769 ,
        0.0065317 , -0.22995059,  0.15351467,  0.37679845,  0.49788803,
        0.29039234,  1.7243502 ,  0.5656652 , -0.16807993,  0.43005645,
       -1.3396119 ,  0.68820405, -0.05464963,  0.20481057, -0.3968754 ,
       -0.6168609 , -0.5249691 ,  0.47609255,  0.7823584 ,  0.00

In [15]:
model.wv.most_similar("版主")

[('水桶', 0.7116859555244446),
 ('署名', 0.7103889584541321),
 ('刪除', 0.6940228939056396),
 ('喉嚨', 0.6444372534751892),
 ('版面', 0.6238248348236084),
 ('諒', 0.6128705739974976),
 ('Yahoo', 0.608859121799469),
 ('該報', 0.6068049073219299),
 ('申領', 0.6050544381141663),
 ('朱凱翔', 0.6018275022506714)]

In [23]:
len(model.wv.key_to_index.keys())

41592

In [22]:
model.wv.key_to_index.keys()

dict_keys(['的', '是', '在', '了', '--', '我', '有', '也', '都', '※', '就', '人', '不', '他', '會', '2019', '要', '說', '被', '編輯', '新聞', '為', '臺灣', '完整', '台灣', '嗎', '到', '但', '你', '月', '沒有', '記者', '很', '1', '3', '與', '或', '年', '和', '跟', '2', '什麼', '表示', '後', '上', '一個', '可以', '中', '對', '她', '好', '八卦', '日', '讓', '07', '看', '有沒有', '中國', '4', '去', '5', '6', '請', '10', '啊', '就是', '媒體', '自己', '08', '韓國', '香港', '現在', '沒', '時', '09', '大家', '這樣', '又', '等', '我們', '這', '報導', '個', '還是', '瑜', '█', '總統', '大', '才', '想', '不是', '因為', '知道', '連結', '而', '多', '網址', '及', '啦', '他們', '如果', '來源', '短', '真的', '以', '把', '署名', '美國', '前', '不會', '／', '更', '刪除', '今天', '做', '看到', '來', '怎麼', '呢', '能', '用', '水桶', '可能', '這個', '所以', '應該', '那', '政治', '蔡', '內文', '日本', '再', '20', '已經', '備註', '新聞標題', '卻', '吃', '覺得', '吧', '陳', '注意', '從', '還', '發現', '30', '柯文', '12', '8', '問題', '如', '11', '出來', '一', '是不是', '政府', '還有', '最', '台北', '並', '市長', '台', '很多', '將', '者', '14', 'from', '小', '已', '英文', '下', 'on', '開始', '不要', '7', '不過', '國家', 'Sent', '不能',

In [24]:
df["content"][1]

'\n自由時報\n\n比基尼女郎蘭嶼海邊戲水 被po網指褻瀆文化 引發熱議\n\n2019-07-30 13:03\n\n［記者黃明堂／台東報導］昨天有網友在臉書「台東大小事」貼出蘭嶼海邊有比基尼女郎\n的畫面，批評穿「三角內褲」褻瀆神聖的飛魚祭場，此文引發熱議，有人籲遊客自重，也\n有很多人認為傳統文化不應無限上綱，且隨意拍遊客引公審，很不恰當。\n\n「你們當蘭嶼是墾丁嗎？這樣隨便穿著比基尼環島遊，還毫無顧忌的在部落灘頭穿著曝露\n泳裝恣意妄為，簡直褻瀆我們神聖的傳統飛魚祭場公共灘頭。」\n\n網友po文批判，還貼上數張照片，顯示有3位女子在停放拼板舟的海邊戲水，當中有人穿\n比基尼泳裝。\n\n原po指蘭嶼人的文化習俗雖然沒有保守到食古不化的地步，但也非開放到可以接受穿著曝\n露的趴趴走。\n\n此文引發熱烈討論，有站在原po角度，要遊客給族人單純的尊重，也籲島上的民宿多加宣\n導教育。\n\n但也有蘭嶼人說不記得不能穿比基尼到沙灘或者碼頭 ，但不可隨意碰拼板舟。\n\n但有更多聲音認為傳統文化被無限上綱，更有網友嗆「只准自己穿丁字褲，不許遊客穿比\n基尼」。有人諷刺道：「請尊重我們蘭嶼文化，以後到海邊男士請着西裝，女士着禮服」\n。\n\n有網友認為，海邊屬於公共開放場所，去海邊玩玩水，穿著泳裝也並無不妥，有禁忌的部\n分應該只是祭典用的拼板舟，別將事件無限上綱，弄得好像是不喜歡人到蘭嶼，且未經允\n許po遊客的照片已經明顯違反肖像權，侵害人格權小心被當事人吉。\n\nhttp://tinyurl.com/y6cqu429\n\n全文連結:https://tinyurl.com/y2xr9khe\n\n--\n'

In [25]:
from jieba.analyse import extract_tags
import numpy as np
article = df["content"][1]
tags = extract_tags(article,  None, withWeight=True)
total = np.zeros((100, ))
norm = 0.
for t, w in tags:
    if t in model.wv:
        total = total + w * model.wv[t]
        norm = norm + w
print("關鍵字平均:")
print(total / norm)

關鍵字平均:
[-0.18199063 -0.14195941  0.19621513  0.15789902 -0.03953427 -0.40006074
  0.23784607  0.48387173 -0.12175655  0.06115608 -0.01285208 -0.25287613
  0.18768806  0.29471902  0.05427412 -0.3335811   0.03800972 -0.18782404
  0.06801197 -0.20680142 -0.00974664  0.03179341  0.40890523 -0.22519745
  0.32917984 -0.31908238 -0.15232757  0.03275971 -0.17374606 -0.13797333
  0.1009906   0.13647768  0.23830207 -0.00668812  0.08047296  0.25727789
  0.12894386 -0.10715789  0.10936423 -0.28819162 -0.11137065 -0.43696134
 -0.11247133  0.23094068  0.27542359  0.0029047  -0.24226879  0.18804082
  0.23217301  0.24839255  0.05124593 -0.20525119 -0.04892838  0.2216484
 -0.22530024  0.28789341  0.23262868  0.14816796 -0.10423589  0.04942471
  0.20914679  0.08577386 -0.02197928  0.02527252 -0.0512985   0.13475787
  0.0613504   0.29060532 -0.1596146   0.30802921 -0.2379231   0.14244094
  0.25465911 -0.06473936  0.20773948  0.19007015 -0.00129664 -0.23573658
  0.20510515  0.12775467 -0.00491852 -0.03811

In [26]:
def docvec(article):
    tags = extract_tags(article, None, withWeight=True)

    total = np.zeros((100, ))
    norm = 0.
    for t, w in tags:
        if t in model.wv:
            total = total + w * model.wv[t]
            norm = norm + w
    return total / norm
dv = df["content"].apply(docvec)
dv

0        [-0.18072148374142605, -0.16200423090848293, -...
1        [-0.18199062572777586, -0.14195940872487955, 0...
2        [-0.06018132281721625, -0.04604681188712372, 0...
3        [-0.5882152400914854, -0.28179746748594786, 0....
4        [-0.14179631919727537, 0.045539942241038196, -...
                               ...                        
19995    [-0.3782272782891058, 0.16303221322978223, -0....
19996    [-0.22958999579786055, -0.014952021657634335, ...
19997    [-0.24782929476922527, -0.08923092335200795, 0...
19998    [-0.4359548618117863, -0.11272773135891591, 0....
19999    [-0.06282105844648983, 0.08384834218801661, 0....
Name: content, Length: 20000, dtype: object

In [27]:
# 準備寫入我們的向量
f = open('article.txt' ,'w')
f.write('{} {}\n'.format(len(dv), 100))
for i, v in enumerate(dv):
    s = ' '.join(map(str, list(v)))
    f.write('{} {}\n'.format(i, s))
f.close()

In [28]:
import gensim
d2v = gensim.models.KeyedVectors.load_word2vec_format('./article.txt', binary=False)

In [31]:
i = input("選擇一篇文章:")
print("原文標題:", df.iloc[int(i)]["title"])
print("原文內容:", df.iloc[int(i)]["content"])

for si, sv in d2v.most_similar(i, topn=2):
    print("*" * 50)
    print("相似度:", sv)
    print("推薦標題:", df.iloc[int(si)]["title"])
    print("推薦內容:", df.iloc[int(si)]["content"])

選擇一篇文章:10000
原文標題: 川普3000億加稅話音剛落 人民幣開盤暴跌4
原文內容: 
川普3000億加稅話音剛落 人民幣開盤暴跌400點

https://www.ntdtv.com/b5/2019/08/02/a102635915.html

【新唐人北京時間2019年08月02日訊】美中最新一輪貿易談判無果而終，川普（特朗普）
總統8月1日連發多篇推文，指責中共不守信用，並下令對3000億美元中國輸美商品加徵
10%的關稅。次日，人民幣兌美元匯率開盤即暴跌超400點，在岸人民幣一度跌破6.95關口
，離岸人民幣觸及去年10月末以來最低位6.97。

貿易戰再升級 人民幣暴跌

最新一輪貿易談判7月31日在上海落幕，沒有任何突破，雙方表示9月在美國繼續磋商。8
月1日，川普連發4則推文，歷數中共未兌現的數個承諾，並宣布美方將對餘下3,000億美
元中國輸美商品徵稅。

川普在推文中稱，「最近，中國（共）同意從美國大量購買農產品，但卻沒有做。此外，
我的朋友習主席說，他將阻止向美國出售芬太尼——這也從未實踐，許多美國人繼續死亡
！」

他表示，貿易談判會繼續，美國將於9月1日開始，對來自中國的剩餘3000億美元的進口商
品加徵10％的關稅。

川普話音剛落，8月2日，人民幣對美元匯率開盤即暴跌。在岸人民幣跳空低開，下挫逾
400點，一度跌破6.95關口；離岸人民幣對美元震盪下行逾百點，盤中最低觸及2018年10
月末以來低位6.9752。截至9點38分，在岸、離岸人民幣對美元分別為6.9396、6.9608。

同時，人民幣對美元中間價調降58個基點，報6.8996，本週累計調降200個基點。


分析：人民幣破7是大概率

大多數分析家認為，美中貿易戰前景不樂觀，今年人民幣破7（即1美元兌人民幣匯率超過
7元）是大概率之事。

人民幣是否破7，一直是業界關注的焦點。瑞銀此前預計，如果美中貿易戰進一步升級，
今年人民幣兌美元匯率恐逼近7.5。

摩根大通在去年9月的報告中表示，如果美國在2019年對所有中國商品加徵25%關稅，預計
人民幣匯率在今年9月下探7.19。

德意志銀行則認為，人民幣匯率會在今年跌至7.4。

國際投行野村證券在7月8日的報告中，預計美國將在今年底前對剩下的3000億美元中國商
品加徵25%的關稅，而人民幣兌美元匯率年底前也會跌

In [33]:
df

,type,title,content,cut
0,問卦,萬一被女同事看到姆咪 該怎麼解釋？,\n\n如題\n\n只是萬一啦\n\n真的有那麼一個萬一\n\n被女同事看到我手機裡的姆咪圖...,\n \n 如 題 \n \n 只是 萬一 啦 \n \n 真的 有 那麼 一個 萬一 \n...
1,新聞,比基尼女郎蘭嶼海邊戲水 被po網指褻瀆,\n自由時報\n\n比基尼女郎蘭嶼海邊戲水 被po網指褻瀆文化 引發熱議\n\n2019-0...,\n 自由 時報 \n \n 比基尼 女郎 蘭嶼 海邊 戲水 被 po 網指 褻瀆 文化 引...
2,新聞,BBC介紹布袋戲大師陳錫煌 致力傳統藝術年,\n中央社\n\nBBC介紹布袋戲大師陳錫煌 致力傳統藝術年輕化\n\nhttps://ww...,\n 中央社 \n \n BBC 介紹 布袋戲 大師 陳 錫煌 致力 傳統 藝術 年輕化 \...
3,問卦,報案怎麼那麼麻煩？,\n如標題\n\n陪朋友去報案\n\n順便自己報案\n\n發現啊\n\n去警察局報案時\n\...,\n 如 標題 \n \n 陪 朋友 去 報案 \n \n 順便 自己 報案 \n \n 發...
4,問卦,受黨國教育的老人喜歡罵大學生素質低？,\n\n就是啊\n\n受黨國教育的9.2韓粉老人，超喜歡罵現在年輕人素質低落的耶！\n\n在...,\n \n 就是 啊 \n \n 受 黨國 教育 的 9.2 韓粉 老人 超 喜歡 罵 現在...
...,...,...,...,...
19995,問卦,禿子為什麼都會禿到後腦勺？,\n本木村從高中開始\n就留著一頭長捲髮\n這輩子沒有禿頭的煩惱\n家族基因都沒人禿頭\n\...,\n 本 木村 從 高中 開始 \n 就 留著 一頭 長捲髮 \n 這輩子 沒有 禿頭 的 ...
19996,新聞,選舉激情過後 明年電價漲不漲？,\n2019-09-18 20:02經濟日報 記者江睿智╱即時報導\n\n如外界預期，經濟部...,\n 2019 09 18 20 02 經濟日報 記者 江 睿智 ╱ 即時 報導 \n \n...
19997,問卦,日本有人提議老人應放棄選舉權,\n\n莫邦富的日本管窺（258)日本有人提議老人應放棄選舉權\n\n2019/07/26\...,\n \n 莫邦富 的 日本 管窺 258 日本 有人 提議 老人 應 放棄 選舉權 \n ...
19998,問卦,頂呱呱哪一樣最好吃?,\n明天頂瓜瓜單品8折\n\n小魯很少吃，一年就吃這一次\n\n已經煩惱到晚餐吃不下東西了\...,\n 明天 頂瓜瓜 單品 8 折 \n \n 小魯 很少 吃 一年 就 吃 這 一次 \n ...


In [36]:
i = input("選擇一篇文章:")
print("原文標題:", df.iloc[int(i)]["title"])
print("原文內容:", df.iloc[int(i)]["content"])
print("*" * 50)
for si, sv in d2v.most_similar(i):
    print("相似度:", sv)
    print("推薦標題:", df.iloc[int(si)]["title"])
    #print("推薦內容:", df.iloc[int(si)]["content"])

選擇一篇文章:15000
原文標題: 我國四科大辦首批2+i專班 印尼來台訪視
原文內容: 
1.媒體來源:
※ 聯合新聞網

2.記者署名
※ 聯合報 記者潘乃欣

3.完整新聞標題:
※ 我國四科大辦首批2+i專班 印尼來台訪視成效

4.完整新聞內文:
印尼科研高教部部長哈瑞（Hari Purwanto）應教育部邀請，今年7月15日至20日率團來台，
訪視四所科大辦理「印尼2+i技職產學合作專班」成效。教育部表示，印尼預計增設百所技
職院校，目前缺乏大量技職師資，將進一步推動獎助2+i專班印尼學生繼續進修碩士，返國
後成為講師。

教育部表示，「印尼2+i技職產學合作專班」為台、印官方技職教育新合作模式，由印尼科
研高教部選送成績優良的國立技職學院二專、三專畢業生參加甄選，我方學校組團至印尼面
試學生來台就讀。醒吾、正修、明新、健行科技大學四校試辦首批專班，88名學生今年3月
27日已來台就學。

教育部說，學生在我國科技大學就讀2年後，安排於實習機構進行半年至1年的專業實習，畢
業後取得我國大學學位。我國科大提供相當於第一年學雜費全免的獎學金，印尼政府提供學
生推薦信、來台機票及1千萬印尼盾（約新台幣2萬4千元）生活費。

教育部表示，印尼預計增設百所技職院校，目前缺乏大量技職師資，將加強和我國合作辦理
師資訓練、鼓勵技職教師赴台進修博碩士學位，也將進一步推動獎助2+i專班印尼學生繼續
進修碩士，返國後成為講師。

教育部次長劉孟奇建議，未來雙方可加強專業領域研究合作，例如指派印尼技職校長前往我
國科大進行智慧農業等領域的創新合作交流，共創產業、學校、學生三贏新局。

5.完整新聞連結 (或短網址):
※ https://udn.com/news/story/7266/3952649

6.備註:
※ 取代台科、北科、雲科、高科的新四科大

--

**************************************************
相似度: 0.9665331840515137
推薦標題: 郭台銘推動AI教育有成　全台108所高中新學期教《人工智慧導
相似度: 0.961122453212738
推薦標題: 新北衛生教育宣導　榮獲全國6都第1名殊榮
相似度: 0.9595509171485901
推薦標題: 陳其邁明參訪 Gogoro 工廠